In [ ]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent / "src"))

import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
commits = pd.read_csv('commits.csv', index_col=0, parse_dates=['date'])
df = commits.sort_values(by='date').reset_index().drop(columns=['index'])

In [ ]:
from private import email_dict
df = df.replace({'email': email_dict})

In [ ]:
df['weight'] = df['ins'] + df['del']
df['yw'] = df['date'].dt.isocalendar().year * 100 + df['date'].dt.isocalendar().week
df1 = df.drop(columns=['date', 'ins', 'del', 'dir', 'path']).groupby(['email', 'yw']).sum().reset_index()

In [ ]:
# Fill empty indexes
emails = df1['email'].unique()
weeks = list(range(df['yw'].min(), df['yw'].max() + 1))
index_template = pd.MultiIndex.from_product([emails, weeks], names=["email", "yw"])
df_template = pd.DataFrame(index = index_template).reset_index()
df_template['zero'] = 0
df2 = df_template.set_index(keys=['email', 'yw']).join(df1.set_index(keys=['email', 'yw'])).fillna(0)
df2['weight'] = df2['weight'] + df2['zero']
df2 = df2.reset_index()
df2

In [ ]:
import datetime
df2['year'] = df2['yw'] // 100
df2['week'] = df2['yw'] % 100
df2['date'] = df2.apply(lambda row: datetime.datetime.strptime("{}-{}-7".format(row['year'], row['week']), "%G-%V-%u"), axis=1)

In [ ]:
df2['moving'] = df2.groupby(['email']).rolling(7, 1)['weight'].mean().droplevel(level=[0])

In [ ]:
fig = px.line(df2, x='date', y='weight', color='email', markers=True)
fig.show()

In [ ]:
fig = px.line(df2, x='date', y='moving', color='email', markers=True)
fig.show()